In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementClickInterceptedException
import time
import warnings
warnings.filterwarnings('ignore')
import re


In [2]:
# connecting to the driver
driver = webdriver.Chrome(r"C:\Users\LENOVO\Downloads\chromedriver.exe")

In [3]:
# opening the website
website = "https://www.booking.com/index.en-gb.html?aid=397646&label=yho748jc-1DCAEoggI46AdIM1gDaGyIAQGYAQm4ARfIAQzYAQPoAQGIAgGoAgO4Ap3Y96YGwAIB0gIkY2NlMDIyZmItZDdlZi00MWVkLWE5YzUtNmIzNmE1NWI"
driver.get(website)
time.sleep(5)

In [4]:
# XPath for the close button of the pop-up
try:
    close_button = driver.find_element(By.XPATH, '//div[@class="ffd93a9ecb dc19f70f85 eb67815534 e91f709929"]/div')
    close_button.click()
except (NoSuchElementException, ElementClickInterceptedException):
    # Handle the case when the close button is not found or cannot be clicked
    print("Failed to close the pop-up")

Failed to close the pop-up


In [5]:
#Scroll to the bottom of the page
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [6]:
# clicking the cities tab
driver.find_element(By.XPATH, '//div[@id="footer_links"]/div[1]/ul/li[3]/a').click()
time.sleep(10)

In [7]:
# Click to close the pop-up
driver.find_element(By.XPATH, '//button[@class="bui-popover__close"]').click()

We have to scrape data for the following cities--
1 Dubai, United Arab Emirates
2 London, United Kingdom
3 Kuala Lumpur, Malaysia
4 Manchester
5 New Delhi
6 Birmingham
7 Berlin
8 Sydney
9 Melbourne
10  Paris
11 Tokyo
12 Toronto

In [8]:
# Define the list of city names you're interested in
cities_of_interest = ["Dubai", "London", "Kuala Lumpur","Manchester", "New Delhi", "Birmingham", "Berlin",
                      "Sydney","Melbourne","Paris", "Tokyo", "Toronto"]

# Find all the city name elements
name_elements = driver.find_elements(By.XPATH, '//div[@class="block_header"]/h2/a')

# Initialize a list to store the city names
name = []

for element in name_elements:
    city_name = element.text
    # Manipulate the text to extract the city name correctly
    city_name = city_name.strip('Hotels in ').split(',')[0]
    name.append(city_name)

In [9]:
# finding the url for cities_of_interest

# Initialize a list to store city URLs
cities_url = []

# Find the URLs for cities of interest
for city_name, city_url in zip(name, driver.find_elements(By.XPATH, '//div[@class="block_header"]/h2/a')):
    if city_name in cities_of_interest:
        cities_url.append(city_url.get_attribute('href'))

In [10]:
cities_url = cities_url[:12]
cities_url

['https://www.booking.com/city/ae/dubai.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/gb/london.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/au/sydney.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/au/melbourne.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/fr/paris.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/de/berlin.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c00874437fd345a43961cf114b437718&inac=0&',
 'https://www.booking.com/city/gb/manchester.en-gb.html?aid=397646&label=yho748jc-1FmAEJ2AEB-AENqAID2AIE&sid=c0

In [11]:
all_hotel_urls = []
for url in cities_url:
    driver.get(url)
    
    # Wait for the page to load successfully
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    # Click to load hotel details
    driver.find_element(By.XPATH, '//div[@class="lp-bui-section bui-spacer--largest x2"]').click()
    
    # select the calender dates
    try:
        checkin = driver.find_element(By.XPATH, '//div[@class="d4babb55ef"]/div[1]/table/tbody/tr[4]/td[3]/span/span')
        checkin.click()
    
        checkout = driver.find_element(By.XPATH, '//div[@class="d4babb55ef"]/div[1]/table/tbody/tr[4]/td[4]/span/span')
        checkout.click()
    
        search = driver.find_element(By.XPATH, '//div[@class="e22b782521 d12ff5f5bf"]')
        search.click()
    
    except (NoSuchElementException, ElementClickInterceptedException):
        # Handle the case when the close button is not found or cannot be clicked
        print("Failed to click")
    
    # Loop through multiple pages (up to 40 pages)
    for page in range(20):
        try:
                
            # extracting url of hotel
            url_elements = driver.find_elements(By.XPATH, '//div[@class="aca0ade214 aaf30230d9 cd2e7d62b0 b0db0e8ada"]/div/div/div/div/div/div/h3/a')
            for element in url_elements:
                hotel = element.get_attribute('href')
                all_hotel_urls.append(hotel)
                
            
            # click the next page button
            try:
                next_button = driver.find_element(By.XPATH, '//div[@class="ef2dbaeb17"]/div[3]/button/span/span')
                next_button.click()
            except NoSuchElementException:
                # If the "Next" button is not found, break out of the loop
                break
        except StaleElementReferenceException:
            # Handle StaleElementReferenceException by re-locating the element
            continue
    
    

Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click
Failed to click


In [12]:
# printing the lengths of the above scraped list

print(len(all_hotel_urls))

5811


In [13]:
# Create an empty list to store the dictionaries for each hotel
hotel_data_list = []

In [14]:
for i in all_hotel_urls:
    driver.get(i)
    time.sleep(5)
    
    # Create a dictionary to store data for the current hotel
    hotel_data = {}
    
    try:
        hotel = driver.find_element(By.XPATH, '//h2[@class="d2fee87262 pp-header__title"]')
        hotel_data['Hotel_name'] = hotel.text
    except NoSuchElementException:
        hotel_data['Hotel_name'] = '-'
        
    try:
        city = driver.find_element(By.XPATH, '//div[@class="e000754250"]/input')
        hotel_data['City'] = city.get_attribute("value")
    except NoSuchElementException:
        hotel_data['City'] = '-'
    
    try:
        loc = driver.find_element(By.XPATH, '//p[@class="address address_clean"]/span[1]')
        hotel_data['Location'] = loc.text
    except NoSuchElementException:
        hotel_data['Location'] = '-'
        
    try:
        rev = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[2]/span[2]')
        hotel_data['Reviews'] = rev.text.split()[1]
    except NoSuchElementException:
        hotel_data['Reviews'] = '-'
        
    try:
        user = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[2]/span[1]')
        hotel_data['User_rating'] = user.text
    except NoSuchElementException:
        hotel_data['User_rating'] = "-"
        
    try:
        star = driver.find_elements(By.XPATH, '//span[@class="fcd9eec8fb d31eda6efc c25361c37f"]')
        hotel_data['Star_rating'] = len(star)
    except NoSuchElementException:
        hotel_data['Star_rating'] = "-"
        
    try:
        score = driver.find_element(By.XPATH,'//div[@class="daaa8ff09f f62fdb6838"]/div/div[1]')
        hotel_data['Review_score'] = score.text
    except NoSuchElementException:
        hotel_data['Review_score'] = "-"
        
    try:
        wifi = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Free Wi-Fi")]')
        hotel_data['Free_wifi'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Free_wifi'] = 'No'
        
    try:
        room = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Family rooms")]')
        hotel_data['Family_rooms'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Family_rooms'] = 'No'
        
    try:
        smoke = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Non-smoking rooms")]')
        hotel_data['NonSmoking_rooms'] = 'Yes'
    except NoSuchElementException:
        hotel_data['NonSmoking_rooms'] = 'No'
        
    try:
        res = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Restaurent")]')
        hotel_data['Restaurent'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Restaurent'] = 'No'
        
    try:
        bar = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Bar")]')
        hotel_data['Bar'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Bar'] = 'No'
        
    try:
        heat = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Heating")]')
        hotel_data['Heating'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Heating'] = 'No'
        
    try:
        lift = driver.find_element(By.XPATH,'//div[@class="dc6c58be0b"]/span/div/span[contains(text(), "Lift")]')
        hotel_data['Lift'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Lift'] = 'No'
        
    try:
        food = driver.find_element(By.XPATH, '//h4[@class="ph-item-header"][contains(text(),"Breakfast")]')
        
        # Find the elements matching the XPath
        cuisine_elements = driver.find_element(By.XPATH, '//span[@class="ph-item-copy-breakfast-option"]')
    
        # split the text
        cuisine_info = cuisine_elements.text.split()
        
        # Depending on the number of elements obtained after splitting, store them in the respective lists
        if len(cuisine_info) >= 3:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = cuisine_info[1]
            hotel_data['Breakfast_Cuisine3'] = cuisine_info[2]
        elif len(cuisine_info) == 2:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = cuisine_info[1]
            hotel_data['Breakfast_Cuisine3'] = '-'
        elif len(cuisine_info) == 1:
            hotel_data['Breakfast_Cuisine1'] = cuisine_info[0]
            hotel_data['Breakfast_Cuisine2'] = '-'
            hotel_data['Breakfast_Cuisine3'] = '-'
            
    except NoSuchElementException:
        hotel_data['Breakfast_Cuisine1'] = '-'
        hotel_data['Breakfast_Cuisine2'] = '-'
        hotel_data['Breakfast_Cuisine3'] = '-'
        
    try:
        room_type = driver.find_element(By.XPATH,'//span[@class="hprt-roomtype-icon-link "]')
        hotel_data['Room_type'] = room_type.text
    except NoSuchElementException:
        hotel_data['Room_type'] = '-'
        
    try:
        bed1 = driver.find_element(By.XPATH,'//div[@class="hprt-roomtype-bed"]/div/ul[contains(text(), "double")]')
        hotel_data['Double_bed'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Double_bed'] = 'No'
        
    try:
        bed2 = driver.find_element(By.XPATH,'//div[@class="hprt-roomtype-bed"]/div/ul[contains(text(), "single")]')
        hotel_data['Single_bed'] = 'Yes'
    except NoSuchElementException:
        hotel_data['Single_bed'] = 'No'
        
    try:
        price = driver.find_element(By.XPATH, '//span[@class="prco-valign-middle-helper"]')
        hotel_data['Prices'] = price.text
    except NoSuchElementException:
        hotel_data['Prices'] = '-'
        
        
    # Append the hotel_data dictionary to the list
    hotel_data_list.append(hotel_data)

WebDriverException: Message: unknown error: net::ERR_INTERNET_DISCONNECTED
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00456643]
	(No symbol) [0x003EBE21]
	(No symbol) [0x002EDA9D]
	(No symbol) [0x002E9E22]
	(No symbol) [0x002DFCFD]
	(No symbol) [0x002E1101]
	(No symbol) [0x002DFFDD]
	(No symbol) [0x002DF3BC]
	(No symbol) [0x002DF2D8]
	(No symbol) [0x002DDC68]
	(No symbol) [0x002DE512]
	(No symbol) [0x002EF75B]
	(No symbol) [0x00357727]
	(No symbol) [0x0033FD7C]
	(No symbol) [0x00356B09]
	(No symbol) [0x0033FB76]
	(No symbol) [0x003149C1]
	(No symbol) [0x00315E5D]
	GetHandleVerifier [0x006CA142+2497106]
	GetHandleVerifier [0x006F85D3+2686691]
	GetHandleVerifier [0x006FBB9C+2700460]
	GetHandleVerifier [0x00503B10+635936]
	(No symbol) [0x003F4A1F]
	(No symbol) [0x003FA418]
	(No symbol) [0x003FA505]
	(No symbol) [0x0040508B]
	BaseThreadInitThunk [0x75A0344D+18]
	RtlInitializeExceptionChain [0x77919802+99]
	RtlInitializeExceptionChain [0x779197D5+54]


In [15]:
len(hotel_data_list)

2035

In [16]:
%store hotel_data_list

Stored 'hotel_data_list' (list)


In [17]:
hotel_data_list[:3]

[{'Hotel_name': 'Unlock Metropolis Business Bay',
  'City': 'Dubai',
  'Location': 'Marasi Drive 1 AG Tower, Dubai, United Arab Emirates',
  'Reviews': '1,012',
  'User_rating': 'Very good',
  'Star_rating': 4,
  'Review_score': '8.5',
  'Free_wifi': 'No',
  'Family_rooms': 'Yes',
  'NonSmoking_rooms': 'Yes',
  'Restaurent': 'No',
  'Bar': 'Yes',
  'Heating': 'Yes',
  'Lift': 'Yes',
  'Breakfast_Cuisine1': '-',
  'Breakfast_Cuisine2': '-',
  'Breakfast_Cuisine3': '-',
  'Room_type': 'Deluxe Studio with Kitchen and Balcony',
  'Double_bed': 'No',
  'Single_bed': 'No',
  'Prices': '₹ 10,270'},
 {'Hotel_name': 'Element Dubai Airport',
  'City': 'Dubai',
  'Location': 'Airport Road, Al Garhoud, Next to Airport Terminal 1, Garhoud, 10001 Dubai, United Arab Emirates',
  'Reviews': '1,133',
  'User_rating': 'Very good',
  'Star_rating': 0,
  'Review_score': '8.3',
  'Free_wifi': 'No',
  'Family_rooms': 'Yes',
  'NonSmoking_rooms': 'Yes',
  'Restaurent': 'No',
  'Bar': 'Yes',
  'Heating': 'No'